In [1]:
using Toms566
using ForwardDiff
p = Problem()
x0 = p.x0;
y0 = p.y0;

In [10]:
function newtMin(p, x0, method)
    #set up all the variables
    k = 0;  #counter
    kk = 0; #outer loop counter
    f = p.obj(x0);  #f is the objective value at point x
    g = p.grd(x0);  #g is the at point x
    H = p.hes(x0);  #H is the Hessian at point x
    c = p.cons(x0); #c is the constraint at point x
    J = ForwardDiff.jacobian(f); #J is the Jacobian of the function
    tau = norm(g,2);  #calculate the initial 2-norm of the gradient at x0
    rho = 1; #initial value for rho
    zeta = 1; #initial value for zeta (our constraint growth checker)
    
    #We exit after |g| < tauGoal.  Here we scale tauGoal based on the initial value of tau
    if tau > 10e5
        tauGoal = tau*10e-10
    elseif tau > 10e3
        tauGoal = tau*10e-4
    elseif tau > 10e-3
        tauGoal = tau*10e-5
    else
        tauGoal = tau*10e-4
    end
    
    
    if method == "newton"
        alpha = 1;  #a part of the step size for the newton step
        x = x0;  #setting the initial value of x
        mu = 10e-4  #this is for the Armijo backtracking linesearch, it helps define a better condition for an appropriate step size
        sigma = 10e-5  #this is the minimum value we will accept for an eigenvalue inside the Hessian

        #Main loop.  We exit when |g| is less than a predefined value (ie: we are near a stationary point)
        while tau > tauGoal
            if k == 10e3  #if we max iterations, exit and return the current value
                @printf("Maxed iterations")
                return (k, p.obj(x))
            end

            #condition H so that it is definite positive under all conditions
            A = eigvals(H)
            V = eigvecs(H)
            for i in 1:1:length(A)
                if abs(A[i]) >= sigma
                    A[i] = abs(A[i])
                else
                    A[i] = sigma
                end
            end
            A = diagm(vec(A))
            H = *(*(V,A),transpose(V))

            #find search direction
            d = H\-(g)

            #Armijo Backtracking
            alpha = 1;
            while p.obj(x + d*alpha) > (p.obj(x) + alpha*mu*dot(g,d))
                alpha = alpha/1.5;
            end

            #compute next x
            x = x + alpha * d;

            #clean up for next loop
            f = p.obj(x);
            g = p.grd(x);
            H = p.hes(x);
            k = k + 1;
            tau = norm(g,2)
        end
        return (k, p.obj(x))  #return the value we've found
    end
    
    if method == "augLang"
        tau_c = 
        tau_g = 
        
        while norm(c) > tau_c && norm(g - *(J',y)) < tau_g
            if kk == 10e3  #if we max iterations, exit and return the current value
                @printf("Maxed iterations")
                return (k, p.obj(x))
            end
            
            L = f - *(y',c) + (rho/2)*(norm(c,2)^2);
            gx_L = g - *(J',y-rho*c)
            
            while norm(gx_L,2) > tauGoal
                #condition H so that it is definite positive under all conditions
                H_L = ForwardDiff.hessian(L);
                g_L = ForwardDiff.gradient(L);
                A = eigvals(H_L)
                V = eigvecs(H_L)
                
                for i in 1:1:length(A)
                    if abs(A[i]) >= sigma
                        A[i] = abs(A[i])
                    else
                        A[i] = sigma
                    end
                end
                A = diagm(vec(A))
                H_L = *(*(V,A),transpose(V))

                #find search direction
                d = H_L\-(g_L)

                #Armijo Backtracking
                alpha = 1;
                while p.obj(x + d*alpha) > (p.obj(x) + alpha*mu*dot(g_L,d))
                    alpha = alpha/1.5;
                end

                #compute next x
                x = x + alpha * d;

                #clean up for next loop
                c = p.cons(x);
                f = p.obj(x);
                J = ForwardDiff.jacobian(f);
                L = f - *(y',c) + (rho/2)*(norm(c,2)^2);
                H_L = ForwardDiff.hessian(L);
                g_L = ForwardDiff.gradient(L);
                gx_L = g_L - *(J',y-rho*c)
                tau = norm(gx_L,2);
                k = k + 1;
            end                
            
            #update y and rho
            if norm(c,2) < zeta
                rho = rho; #just put here to keep track of algorithm from notes
                y = y - rho*c;
            else
                rho = rho * 10;
                y = y;
            end
            
            kk = kk + 1;
            c = p.cons(x);
            g = p.grd(x);
            J = ForwardDiff.jacobian(f);
        end
        return (k, p.obj(x),p.cons(x))  #return the value we've found, as well as the value of the constraint at that point
end

#run the minimizer
newtMin(p, x0, augLang)

Maxed iterations

(10000,0.9315609411350718)

INFO: Nothing to be done
INFO: METADATA is out-of-date — you may not have the latest version of ForwardDiff
INFO: Use `Pkg.update()` to get the latest versions of your packages
